In [1]:
%matplotlib inline


## Aaron's installation cheatsheet (for linux)
```
conda create -n finrl --no-default-packages
conda activate finrl
git clone git@github.com:AI4Finance-Foundation/FinRL-Meta.git
cd FinRL-Meta
mamba install -y python==3.8 swig
mamba install -y ta-lib pyfolio -c conda-forge
mamba install -y pytorch -c pytorch
mamba install tensorboard
pip install -e .
```

The last line allows you to edit the python files without having to redo the pip install

# Imports

In [2]:
import os
import pandas as pd
from collections import defaultdict

from meta import config, config_tickers
# from meta.env_portfolio_allocation.env_portfolio_yahoofinance import StockPortfolioEnv
from meta.env_portfolio_allocation.env_portfolio_yahoofinance import StockPortfolioEnv
from meta.data_processor import DataProcessor
import numpy as np
import cvxpy as cp
import seaborn as sns
import matplotlib.pyplot as plt

from agents.stablebaselines3_models import DRLAgent
from stable_baselines3 import A2C


In [3]:
# function from example notebooks
def data_split(df, start, end, target_date_col="time"):
    """
    split the dataset into training or testing using time
    :param data: (df) pandas dataframe, start, end
    :return: (df) pandas dataframe
    """
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    dates = pd.to_datetime(df[target_date_col])
    data = df[(dates >= start) & (dates < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data


In [4]:
# example markowitz agent using info dict from environment to do optimization at each time step
class MarkowitzAgent:
    """Provides implementations for Markowitz agent (mean-variance optimization)
    Attributes
    ----------
        env: gym environment class
            user-defined class
    Methods
    -------
        prediction()
            make a prediction in a test dataset and get result history
    """

    def __init__(
            self,
            env,
            risk_aversion=10,
            annual_risk_free_rate=0.03 # disregard risk free rate since RL disregards
            ):
        super().__init__()
        self.risk_aversion = risk_aversion
        self.env = env
        # compute daily risk free rate from annual risk free rate
        # self.risk_free_rate = (1 + annual_risk_free_rate) ** (1 / 365) - 1
        # disable risk free rate for now
        self.risk_free_rate = -1

    def get_model(self, model_name, model_kwargs):
        raise NotImplementedError()

    def train_model(self, model, cwd, total_timesteps=5000):
        raise NotImplementedError()

    def act(self, state, info):
        """
        This is the core of markowitz portfolio optimization
        it maximizes the éxpected_return - risk_aversion * risk
        with expected_return = mean_returns @ portfolio_weights
        and risk = portfolio_weights.T @ cov @ portfolio_weights
        The constraints say that the weights must be positive and sum to 1

        returns the action as the weights of the portfolio
        """
        # unpack state to get covariance and means
        data = info["data"].copy()
        # from the data estimate returns and covariances
        cov = data.iloc[-1].cov_list
        mean_returns = data[
            data["time"] == data["time"].max()
        ]["ewm_returns"].to_numpy()

        # solve markowitz model with cvxpy
        # initialize model
        num_stocks = len(mean_returns)
        weights = cp.Variable(num_stocks)
        risk_free_weight = cp.Variable(1)
        # define constraints
        # constraints = [cp.sum(weights) + risk_free_weight ==
        #                1, weights >= 0, risk_free_weight >= 0]
        constraints = [cp.sum(weights) == 1,
                       weights >= 0,
                    #    risk_free_weight >= 0
                       ]
        # define objective
        portfolio_return = mean_returns @ weights # + risk_free_weight*self.risk_free_rate
        portfolio_risk = cp.quad_form(weights, cov)
        # define objective
        objective = cp.Maximize(
            portfolio_return - self.risk_aversion * portfolio_risk)
        # define problem
        problem = cp.Problem(objective, constraints)
        # solve problem
        problem.solve()
        # get weights
        weights = weights.value
        # get action. if using risk free rate then integrate it into the action
        action = weights
        # action = np.concatenate([weights, risk_free_weight.value])
        action = np.maximum(action, 0)
        action = action / np.sum(action)
        return action

    def prediction(self, environment):
        # args = Arguments(env=environment)
        # args.if_off_policy
        # args.env = environment

        # test on the testing env
        state, info = environment.reset()
        day = environment.sorted_times[environment.time_index]
        history = defaultdict(list)

        total_asset = environment.portfolio_value
        history["date"].append(day)
        history["total_assets"].append(total_asset)
        history["episode_return"].append(0)
        # episode_total_assets.append(environment.initial_amount)
        done = False
        while not done:
            action = self.act(state, info)
            state, reward, done, trunc, info = environment.step(action)
            day = environment.sorted_times[environment.time_index]


            total_asset = environment.portfolio_value
            episode_return = total_asset / environment.initial_amount
            history["date"].append(day)
            history["total_assets"].append(total_asset)
            history["episode_return"].append(episode_return)
        print("Test Finished!")
        # return episode total_assets on testing data
        print("episode_return", episode_return)
        return pd.DataFrame(history)

In [5]:
# some hyperparameters for finrl
start_date=config.TRAIN_START_DATE
end_date=config.TRADE_END_DATE
ticker_list=config_tickers.DOW_30_TICKER
time_interval='1D'
data_source='yahoofinance'
technical_indicator_list= config.INDICATORS
if_vix=True
hmax=100
initial_amount=1000000
transaction_cost_pct=0.001
reward_scaling=1e-4
use_cached_model = False


# Download and process data

In [6]:
dp = DataProcessor(
    data_source=data_source,
    start_date=start_date,
    end_date=end_date,
    time_interval=time_interval,
)

price_array, tech_array, turbulence_array = dp.run(
    ticker_list, technical_indicator_list, if_vix=if_vix, cache=True, select_stockstats_talib=0
)

# add covariance matrix as states
df = dp.dataframe
df = df.sort_values(['time', 'tic'], ignore_index=True)
df.index = df.time.factorize()[0]

df["pct_change"] = df.groupby("tic").close.pct_change()

cov_list = []
# look back is one year
lookback = 252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i, :]
    price_lookback = data_lookback.pivot_table(
        index='time', columns='tic', values='close')
    return_lookback = price_lookback.pct_change().dropna()
    covs = return_lookback.cov().values
    cov_list.append(covs)
df["mean_pct_change_lookback"] = df.rolling(lookback)["pct_change"].mean()
df["ewm_returns"] = df["pct_change"].ewm(span=50).mean()
df_cov = pd.DataFrame(
    {'time': df.time.unique()[lookback:], 'cov_list': cov_list})
df = df.merge(df_cov, on='time')
df = df.sort_values(['time', 'tic']).reset_index(drop=True)


yahoofinance successfully connected
Using cached file ./cache/AXP_AMGN_AAPL_BA_CAT_CSCO_CVX_GS_HD_HON_IBM_INTC_JNJ_KO_JPM_MCD_MMM_MRK_MSFT_NKE_PG_TRV_UNH_CRM_VZ_V_WBA_WMT_DIS_DOW_yahoofinance_2014-01-01_2021-12-01_1D.pickle
tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
[*********************100%***********************]  1 of 1 completed
           date       open       high        low      close  adjusted_close  \
0    2014-01-02  14.320000  14.590000  14.000000  14.230000       14.230000   
1    2014-01-03  14.060000  14.220000  13.570000  13.760000       13.760000   
2    2014-01-06  13.410000  14.000000  13.220000  13.550000       13.550000   
3    2014-01-07  12.380000  13.280000  12.160000  12.920000       12.920000  

/home/aaron/Research/finrl_experiments/FinRL-Meta/meta/data_processors/yahoofinance.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(tmp_df)


Data clean for ^VIX is finished.
Data clean all finished!
Successfully transformed into array


# Run markowitz model on test data

In [7]:
test_df = data_split(
    df,
    start=config.TEST_START_DATE,
    end=config.TEST_END_DATE
)

stock_dimension = len(test_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_amount,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": reward_scaling
}
e_test_gym = StockPortfolioEnv(df=test_df, **env_kwargs)
agent = MarkowitzAgent(e_test_gym)
markowitz_history_df = agent.prediction(e_test_gym)
markowitz_history_df["method"] = "markowitz"

Stock Dimension: 30, State Space: 30
begin_total_asset:1000000
end_total_asset:1317065.746292424
Sharpe:  1.8108090198148892
Test Finished!
episode_return 1.317065746292424


# Train RL model and evaluate on test data

In [8]:

train_df = data_split(df, start=config.TRAIN_START_DATE,
                        end=config.TRAIN_END_DATE)
stock_dimension = len(train_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_amount,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": reward_scaling
}
# use cached model if you are iterating on evaluation code
if use_cached_model and os.path.exists("saved_models/a2c_model.pt"):
    trained_a2c = A2C.load("saved_models/a2c_model.pt")
else:
    e_train_gym = StockPortfolioEnv(df=train_df, **env_kwargs)
    agent = DRLAgent(env=e_train_gym)

    A2C_PARAMS = {
        "n_steps": 10,
        "ent_coef": 0.005,
        "learning_rate": 0.0004
    }
    model_a2c = agent.get_model(model_name="a2c", model_kwargs=A2C_PARAMS)
    trained_a2c = agent.train_model(
        model=model_a2c,
        tb_log_name='a2c',
        total_timesteps=40000
        )
    # save trained_a2c model
    trained_a2c.save("saved_models/a2c_model.pt")

# evaluate on test data
test_df = data_split(
    df,
    start=config.TEST_START_DATE,
    end=config.TEST_END_DATE
)
e_test_gym = StockPortfolioEnv(df=test_df, **env_kwargs)

df_daily_return, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment=e_test_gym,
)
df_daily_return["method"] = "a2c"


Stock Dimension: 30, State Space: 30
{'n_steps': 10, 'ent_coef': 0.005, 'learning_rate': 0.0004}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to tensorboard_log/a2c/a2c_5
begin_total_asset:1000000
end_total_asset:1248417.1776510237
Sharpe:  1.774486874366264
begin_total_asset:1000000
end_total_asset:1252205.0777428634
Sharpe:  1.8009995898180302
begin_total_asset:1000000
end_total_asset:1310724.501821081
Sharpe:  2.140183468539528
begin_total_asset:1000000
end_total_asset:1323749.164922317
Sharpe:  2.1562175466180995
begin_total_asset:1000000
end_total_asset:1280735.6538467412
Sharpe:  1.9666605233148007
begin_total_asset:1000000
end_total_asset:1279608.3690670228
Sharpe:  1.9384561262521112
begin_total_asset:1000000
end_total_asset:1277645.2520812936
Sharpe:  1.9393514169745476
begin_total_asset:1000000
end_total_asset:1300578.0002893917
Sharpe:  2.047242121688753
begin_total_asset:1000000
end_total_asset:1286382.7150207304
Sha

begin_total_asset:1000000
end_total_asset:1299695.9573273773
Sharpe:  2.029998116312385
begin_total_asset:1000000
end_total_asset:1287498.13528632
Sharpe:  1.9598355204964573
begin_total_asset:1000000
end_total_asset:1267713.641670693
Sharpe:  1.868875554678957
begin_total_asset:1000000
end_total_asset:1319100.2995866975
Sharpe:  2.133155268567836
begin_total_asset:1000000
end_total_asset:1270874.827651288
Sharpe:  1.880764376270588
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 93       |
|    ep_rew_mean        | 1.14e+08 |
| time/                 |          |
|    fps                | 201      |
|    iterations         | 400      |
|    time_elapsed       | 19       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -42.4    |
|    explained_variance | 0        |
|    learning_rate      | 0.0004   |
|    n_updates          | 399      |
|    policy_loss        | 2.83e+08 |
|    rew

begin_total_asset:1000000
end_total_asset:1264323.4103527877
Sharpe:  1.8741667546599874
begin_total_asset:1000000
end_total_asset:1280548.1593756527
Sharpe:  1.963204656770442
begin_total_asset:1000000
end_total_asset:1277123.2424829819
Sharpe:  1.9216512518864424
begin_total_asset:1000000
end_total_asset:1296892.0423589568
Sharpe:  2.069219893215755
begin_total_asset:1000000
end_total_asset:1331402.01574263
Sharpe:  2.194129399661603
begin_total_asset:1000000
end_total_asset:1274022.1862170773
Sharpe:  1.8784874029338607
begin_total_asset:1000000
end_total_asset:1289858.6506700385
Sharpe:  2.0085203588964307
begin_total_asset:1000000
end_total_asset:1298276.012002997
Sharpe:  2.0616142011937906
begin_total_asset:1000000
end_total_asset:1282316.4083629395
Sharpe:  1.9685286301342675
begin_total_asset:1000000
end_total_asset:1309561.896911672
Sharpe:  2.1320715625123077
begin_total_asset:1000000
end_total_asset:1302299.4838221138
Sharpe:  2.059898002880881
-----------------------------

begin_total_asset:1000000
end_total_asset:1274972.7634259458
Sharpe:  1.8918613738232803
begin_total_asset:1000000
end_total_asset:1271109.313701045
Sharpe:  1.8883957565508631
begin_total_asset:1000000
end_total_asset:1297284.898826931
Sharpe:  2.010775321593004
begin_total_asset:1000000
end_total_asset:1296761.3392085466
Sharpe:  2.011083558804591
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.14e+08  |
| time/                 |           |
|    fps                | 202       |
|    iterations         | 1100      |
|    time_elapsed       | 54        |
|    total_timesteps    | 11000     |
| train/                |           |
|    entropy_loss       | -42.3     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updates          | 1099      |
|    policy_loss        | 2.65e+08  |
|    reward             | 1203661.6 |
|    std                | 0.99      |
| 

begin_total_asset:1000000
end_total_asset:1308134.6525610262
Sharpe:  2.0622915092890017
begin_total_asset:1000000
end_total_asset:1288694.3970145627
Sharpe:  1.9815876591901047
begin_total_asset:1000000
end_total_asset:1274718.06541457
Sharpe:  1.8905786126063462
begin_total_asset:1000000
end_total_asset:1302999.168247715
Sharpe:  2.068908449805643
begin_total_asset:1000000
end_total_asset:1287211.6485775139
Sharpe:  1.9599625219542502
begin_total_asset:1000000
end_total_asset:1318102.029555186
Sharpe:  2.1556351941571745
begin_total_asset:1000000
end_total_asset:1290929.011676243
Sharpe:  1.979967994705965
begin_total_asset:1000000
end_total_asset:1306817.4520171785
Sharpe:  2.055763407197912
begin_total_asset:1000000
end_total_asset:1289432.2276075247
Sharpe:  1.9494931394913397
begin_total_asset:1000000
end_total_asset:1288983.785301512
Sharpe:  1.9542972141994903
begin_total_asset:1000000
end_total_asset:1317716.8272025266
Sharpe:  2.123398802236188
-------------------------------

...

begin_total_asset:1000000
end_total_asset:1279866.7526098252
Sharpe:  1.9468232396825722
begin_total_asset:1000000
end_total_asset:1295022.7096499398
Sharpe:  2.014026120122378
begin_total_asset:1000000
end_total_asset:1270132.4160869517
Sharpe:  1.8988290071268907
begin_total_asset:1000000
end_total_asset:1292484.5157669247
Sharpe:  2.013193104051534
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.14e+08  |
| time/                 |           |
|    fps                | 202       |
|    iterations         | 2500      |
|    time_elapsed       | 123       |
|    total_timesteps    | 25000     |
| train/                |           |
|    entropy_loss       | -41.9     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updates          | 2499      |
|    policy_loss        | 2.92e+08  |
|    reward             | 1278172.8 |
|    std                | 0.98      |


begin_total_asset:1000000
end_total_asset:1312302.353503106
Sharpe:  2.1302227996216447
begin_total_asset:1000000
end_total_asset:1289975.1528904284
Sharpe:  2.023877935701507
begin_total_asset:1000000
end_total_asset:1314629.0808761285
Sharpe:  2.149792760443045
begin_total_asset:1000000
end_total_asset:1292045.3365561224
Sharpe:  2.010261163765017
begin_total_asset:1000000
end_total_asset:1295411.035105161
Sharpe:  2.03949047932455
begin_total_asset:1000000
end_total_asset:1287358.023836278
Sharpe:  1.994525896965811
begin_total_asset:1000000
end_total_asset:1283975.916513171
Sharpe:  1.9720019413980279
begin_total_asset:1000000
end_total_asset:1299128.7964866608
Sharpe:  2.0593964614419917
begin_total_asset:1000000
end_total_asset:1289084.4139824447
Sharpe:  1.984813878597619
begin_total_asset:1000000
end_total_asset:1286004.6490612854
Sharpe:  2.0010688195406106
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep

begin_total_asset:1000000
end_total_asset:1302820.2850248334
Sharpe:  2.0681043542063797
begin_total_asset:1000000
end_total_asset:1290629.1814706337
Sharpe:  2.0044079999760673
begin_total_asset:1000000
end_total_asset:1285465.023247714
Sharpe:  1.9802032032089478
begin_total_asset:1000000
end_total_asset:1303806.8404390777
Sharpe:  2.1097562194610924
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.14e+08  |
| time/                 |           |
|    fps                | 203       |
|    iterations         | 3200      |
|    time_elapsed       | 157       |
|    total_timesteps    | 32000     |
| train/                |           |
|    entropy_loss       | -41.8     |
|    explained_variance | 5.96e-08  |
|    learning_rate      | 0.0004    |
|    n_updates          | 3199      |
|    policy_loss        | 2.91e+08  |
|    reward             | 1092060.8 |
|    std                | 0.974     |

begin_total_asset:1000000
end_total_asset:1287389.0115217275
Sharpe:  2.021961332298677
begin_total_asset:1000000
end_total_asset:1318345.6639241946
Sharpe:  2.187945850629503
begin_total_asset:1000000
end_total_asset:1297680.8133880391
Sharpe:  2.077935373149008
begin_total_asset:1000000
end_total_asset:1335192.6459156077
Sharpe:  2.2632023467875677
begin_total_asset:1000000
end_total_asset:1296443.959252523
Sharpe:  2.061457508254743
begin_total_asset:1000000
end_total_asset:1284780.5259640708
Sharpe:  1.990296019990639
begin_total_asset:1000000
end_total_asset:1288286.2417020358
Sharpe:  2.0095886475606437
begin_total_asset:1000000
end_total_asset:1277967.9590257208
Sharpe:  1.9466616657601634
begin_total_asset:1000000
end_total_asset:1296830.0589498833
Sharpe:  2.0690468187901203
begin_total_asset:1000000
end_total_asset:1299790.006539923
Sharpe:  2.077320059178233
begin_total_asset:1000000
end_total_asset:1302452.6876981268
Sharpe:  2.087028781867444
------------------------------

begin_total_asset:1000000
end_total_asset:1277301.974912057
Sharpe:  1.9545340435579093
begin_total_asset:1000000
end_total_asset:1311842.640527063
Sharpe:  2.117585881680284
begin_total_asset:1000000
end_total_asset:1280728.3499185082
Sharpe:  1.9635179397025257
begin_total_asset:1000000
end_total_asset:1308116.2859572147
Sharpe:  2.117995502140141
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.14e+08  |
| time/                 |           |
|    fps                | 203       |
|    iterations         | 3900      |
|    time_elapsed       | 191       |
|    total_timesteps    | 39000     |
| train/                |           |
|    entropy_loss       | -41.7     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updates          | 3899      |
|    policy_loss        | 2.71e+08  |
|    reward             | 1185502.4 |
|    std                | 0.971     |
| 

In [9]:
df_daily_return["total_assets"] = (1+df_daily_return["daily_return"]).cumprod() * e_test_gym.initial_amount

# Plot returns comparison

In [10]:
cols = ["date", "total_assets", "method"]
df_to_plot = pd.concat([df_daily_return[cols], markowitz_history_df[cols]])

sns.lineplot(
    x="date",
    y="total_assets",
    hue="method",
    data=df_to_plot
    )
plt.title("return comparison")
plt.show()

# Get returns tearsheet using pyfolio
note that this requires a modified version of pyfolio which has bugs, ignore the download_data error as long as you get 
"yahoofinance successfully connected"

In [11]:
# code for analyzing returns using pyfolio, creating a tear sheet
import pyfolio as pf

In [12]:
try:
    _ = dp.processor.download_data(["^GSPC"], save_path="./data/sp500.csv")
except:
    pass

[*********************100%***********************]  1 of 1 completed
           date         open         high          low        close  \
0    2014-01-02  1845.859985  1845.859985  1827.739990  1831.979980   
1    2014-01-03  1833.209961  1838.239990  1829.130005  1831.369995   
2    2014-01-06  1832.310059  1837.160034  1823.729980  1826.770020   
3    2014-01-07  1828.709961  1840.099976  1828.709961  1837.880005   
4    2014-01-08  1837.900024  1840.020020  1831.400024  1837.489990   
...         ...          ...          ...          ...          ...   
1988 2021-11-23  4678.479980  4699.390137  4652.660156  4690.700195   
1989 2021-11-24  4675.779785  4702.870117  4659.890137  4701.459961   
1990 2021-11-26  4664.629883  4664.629883  4585.430176  4594.620117   
1991 2021-11-29  4628.750000  4672.950195  4625.259766  4655.270020   
1992 2021-11-30  4640.250000  4646.020020  4560.000000  4567.000000   

      adjusted_close      volume    tic  day  
0        1831.979980  308060000

In [13]:
sp500_data = pd.read_csv("./data/sp500.csv")

In [14]:
sp500_data = sp500_data.set_index(pd.to_datetime(sp500_data["date"]))
sp500_data.index = sp500_data.index.tz_localize('America/New_York')
sp500_data['pct_change'] = sp500_data.close.pct_change()
sp500_data = sp500_data.dropna()
# sp500_data.index = sp500_data.index.to_pydatetime()

# print quantstats metrics
ideally we want to fix quantstats or compute the metrics ourselves, but this is just an example

quantstats is broken for me requiring the try, catch

In [15]:
# quantstats tear sheet
for method_name in df_to_plot.method.unique():
    print(f"{method_name} tear sheet")
    port_returns = df_to_plot[df_to_plot["method"] == method_name].drop_duplicates("date").set_index("date")["total_assets"]
    port_returns.index = port_returns.index.tz_localize('America/New_York')

    try:
        pf.create_returns_tear_sheet(port_returns.pct_change().dropna(), benchmark_rets=sp500_data["pct_change"])
    except:
        pass

a2c tear sheet
Entire data start date: 2020-08-04
Entire data end date: 2021-09-30
Backtest months: 13


/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/timeseries.py:718: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/plotting.py:693: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


,Backtest
Annual return,26.6%
Cumulative returns,31.6%
Annual volatility,14.3%
Sharpe ratio,1.73
Calmar ratio,3.14
Stability,0.90
Max drawdown,-8.5%
Omega ratio,1.33
Sortino ratio,2.62
Skew,-0.04


markowitz tear sheet
Entire data start date: 2020-08-04
Entire data end date: 2021-09-30
Backtest months: 13


/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/timeseries.py:718: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/plotting.py:693: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


,Backtest
Annual return,26.7%
Cumulative returns,31.7%
Annual volatility,13.6%
Sharpe ratio,1.81
Calmar ratio,4.62
Stability,0.82
Max drawdown,-5.8%
Omega ratio,1.36
Sortino ratio,2.93
Skew,0.54
